# Space Invaders - Atari (An OpenAI Gym Env)

# Install Dependencies

In [1]:
! pip install tensorflow==2.13.0
! pip install gym
! pip install keras-rl2
! pip install gym[atari]
! pip install gym[accept-rom-license]

# Test Random Environment with OpenAI Gym

In [133]:
import gym
import random

In [134]:
env = gym.make('SpaceInvaders-v4', render_mode='human') #Generate the gym environemt in your local. v4 represents 0 repeat action probability.


height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [135]:
#Getting the action names using unwrarring function
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [99]:
episodes = 5 # means we will be playing 5 games of the Space Invaders
for episode in range(1, episodes+1):
    #Resetting the enviroment
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render() # Rendering our environment
        action = random.choice([0,1,2,3,4,5]) #Assigning randon action(6 action names as above mentioned) for each game until game end
        n_state, reward, done, info, _ = env.step(action) #Extracting the result parameters from each step for an action
        score+= reward # Calculating the score
    print('Episode:{} Score:{}'.format(episode, score))
env.close()


d:\myGithub\Space-Invaders-atarigames\venv.SpaceInvaders\lib\site-packages\gym\utils\passive_env_checker.py:289: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(
d:\myGithub\Space-Invaders-atarigames\venv.SpaceInvaders\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode:1 Score:185.0
Episode:2 Score:35.0
Episode:3 Score:355.0
Episode:4 Score:135.0
Episode:5 Score:90.0


# Creating a Deep Learning Model with Keras

In [136]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D
from tensorflow.keras.optimizers import Adam

In [137]:
def game_model(height, width, channels, actions):
    model = Sequential()
    model.add(Conv2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3, height, width, channels)))
    model.add(Conv2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

# Build Agent with Keras-RL

In [138]:
import tensorflow as tf
from keras import __version__
tf.keras.__version__ = __version__ #As the rl/callbacks.py file has from tensorflow.keras instead of from keras import __version__
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [139]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1, value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, enable_dueling_network=True, dueling_type='avg', nb_actions=actions, nb_steps_warmup=10000)

    #dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    
    return dqn

In [140]:
model = game_model(height, width, channels, actions)
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_27 (Conv2D)          (None, 3, 51, 39, 32)     6176      
                                                                 
 conv2d_28 (Conv2D)          (None, 3, 24, 18, 64)     32832     
                                                                 
 conv2d_29 (Conv2D)          (None, 3, 22, 16, 64)     36928     
                                                                 
 flatten_9 (Flatten)         (None, 67584)             0         
                                                                 
 dense_44 (Dense)            (None, 512)               34603520  
                                                                 
 dense_45 (Dense)            (None, 256)               131328    
                                                                 
 dense_46 (Dense)            (None, 6)               

In [124]:
# Reset environment to get initial observation
observation = env.reset()
# Access the image data (first element of the tuple)
image_data = observation[0]
print("Image data shape:", image_data.shape)

Image data shape: (210, 160, 3)


In [126]:
dqn.fit(env, nb_steps=10, visualize=False, verbose=1)

Training for 10 steps ...
Interval 1 (0 steps performed)


ValueError: Error when checking input: expected conv2d_24_input to have 5 dimensions, but got array with shape (1, 3, 2)